Задачи для Анализа:

1) Загрузить данные о заказах пользователей в приложении по доставке продуктов.

2) Объединить данные из таблиц заказов, данных пользователей и информации о продуктах.

3) Оценить эффективность новой системы рекомендаций приложения, проведя АБ-тест с двумя группами пользователей: группа 0 (старая версия приложения) и группа 1 (новая система рекомендаций).

Описание данных:

ab_users_data: содержит информацию о пользователях, историю заказов и действия пользователей в приложении. 

 - Колонки: user_id, order_id, action, time, date, group.

ab_orders: предоставляет подробную информацию о составе заказов. 

 - Колонки: order_id, creation_time, product_ids.

ab_products: содержит подробную информацию о продуктах. 

 - Колонки: product_id, name, price.

In [42]:
import pandas as pd
from scipy.stats import ttest_ind

In [43]:
ab_users_data = pd.read_csv('ab_users_data.csv', parse_dates = ['time', 'date'])
ab_orders = pd.read_csv('ab_orders.csv', parse_dates = ['creation_time'])
ab_products = pd.read_csv('ab_products.csv')

In [44]:
merge_df = pd.merge(ab_users_data, ab_orders, on = 'order_id')

In [45]:
merge_df['product_ids'] = merge_df['product_ids'].apply(lambda x: [int(item) for item in x.strip('{}').split(',')])
merge_df = merge_df.explode('product_ids')
result_df = pd.merge(merge_df, ab_products, left_on='product_ids', right_on='product_id', how='left')
result_df = result_df.drop(columns=['product_ids', 'product_id'])
result_df

,user_id,order_id,action,time,date,group,creation_time,name,price
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,2022-08-26 00:00:19,сок ананасовый,120.0
1,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,2022-08-26 00:00:19,сок мультифрукт,120.0
2,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,2022-08-26 00:00:19,мука,78.3
3,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,2022-08-26 00:00:19,мандарины,90.4
4,965,1256,create_order,2022-08-26 00:02:21,2022-08-26,1,2022-08-26 00:02:21,сосиски,150.0
...,...,...,...,...,...,...,...,...,...
14564,1461,59487,create_order,2022-09-08 23:29:06,2022-09-08,0,2022-09-08 23:29:06,чай черный листовой,83.5
14565,1461,59487,create_order,2022-09-08 23:29:06,2022-09-08,0,2022-09-08 23:29:06,сок яблочный,120.0
14566,1461,59487,create_order,2022-09-08 23:29:06,2022-09-08,0,2022-09-08 23:29:06,курица,298.4
14567,1376,59533,create_order,2022-09-08 23:41:24,2022-09-08,0,2022-09-08 23:41:24,морс брусничный,190.0


In [59]:
group_0 = result_df.query('group == 0')
group_1 = result_df.query('group == 1')

In [62]:
group_0_orders = group_0.groupby(['user_id', 'creation_time']).agg({'price': 'sum'}).reset_index()
group_1_orders = group_1.groupby(['user_id', 'creation_time']).agg({'price': 'sum'}).reset_index()

group_0_mean_price = group_0_orders.price.mean()
group_1_mean_price = group_1_orders.price.mean()

print('Средняя сумма одного заказа для группы 0:', group_0_mean_price.round(2))
print('Средняя сумма одного заказа для группы 1:', group_1_mean_price.round(2))

Средняя сумма одного заказа для группы 0: 400.2
Средняя сумма одного заказа для группы 1: 390.22


In [77]:
group_0_prices = group_0_orders.price
group_1_prices = group_1_orders.price

t_stat, p_value = ttest_ind(group_0_prices, group_1_prices)

print('t-статистика:', t_stat)
print('p-значение:', p_value)

alpha = 0.05
if p_value < alpha:
    print('Различия статистически значимы, новая система влияет на средний чек')
else:
    print('Различия статистически не значимы, новая система не влияет на средний чек')

t-статистика: 1.128666793265408
p-значение: 0.2591041859424771
Различия статистически не значимы, новая система не влияет на средний чек


Средняя сумма заказа для группы 0 составляет 400.2, а для группы 1 - 390.22. Проведенный t-тест не выявил статистически значимых различий в средних чеках между группами (p-значение = 0.2591). Следовательно, новая система рекомендаций не оказывает влияния на средний чек.

In [95]:
group_0_order = group_0.groupby('user_id').agg({'creation_time': 'count'}).reset_index()
group_1_order = group_1.groupby('user_id').agg({'creation_time': 'count'}).reset_index()

group_0_mean_orders = group_0_.creation_time.mean()
group_1_mean_orders = group_1_.creation_time.mean()

print('Среднее кол-во заказов для одного пользователя для группы 0:', group_0_mean_orders.round(0))
print('Среднее кол-во заказов для одного пользователя для группы 1:', group_1_mean_orders.round(0))

Среднее кол-во заказов для одного пользователя для группы 0: 11.0
Среднее кол-во заказов для одного пользователя для группы 1: 18.0


In [78]:
group_0_order_counts = group_0_order.creation_time
group_1_order_counts = group_1_order.creation_time

t_stat, p_value = ttest_ind(group_0_order_counts, group_1_order_counts)

print('t-статистика:', t_stat)
print('p-значение:', p_value)

alpha = 0.05
if p_value < alpha:
    print('Различия статистически значимы, новая система влияет на количество заказов на пользователя')
else:
    print('Различия статистически не значимы, новая система не влияет на количество заказов на пользователя')

t-статистика: -13.624031571690642
p-значение: 6.108540480706531e-39
Различия статистически значимы, новая система влияет на количество заказов на пользователя


Среднее количество заказов на пользователя для группы 0 составляет 11, а для группы 1 - 18. Проведенный t-тест показал статистически значимые различия (p-значение < 0.05), что говорит о влиянии новой системы рекомендаций на увеличение количества заказов.

In [85]:
group_0_all_money = group_0.price.sum()
group_1_all_money = group_1.price.sum()

print('Вся выручка для группы 0:', group_0_all_money.round(2))
print('Вся выручка для группы 1:', group_1_all_money.round(2))

Вся выручка для группы 0: 643521.7
Вся выручка для группы 1: 979835.6


In [86]:
group_0_total_revenue = group_0.price
group_1_total_revenue = group_1.price

t_stat, p_value = ttest_ind(group_0_total_revenue, group_1_total_revenue)

print('t-статистика:', t_stat)
print('p-значение:', p_value)

alpha = 0.05
if p_value < alpha:
    print('Различия статистически значимы, новая система влияет на общую выручку')
else:
    print('Различия статистически не значимы, новая система не влияет на общую выручку')

t-статистика: 1.731078581117817
p-значение: 0.08345892434326715
Различия статистически не значимы, новая система не влияет на общую выручку


Общая выручка для группы 0 составляет 643521.7, а для группы 1 - 979835.6. Однако, статистический тест не выявил значимых различий в общей выручке между группами (p-значение = 0.0835). Следовательно, новая система рекомендаций не оказывает статистически значимого влияния на общую выручку.

# Основываясь на стат тесты, можно сделать вывод, что новая система рекомендаций оказывает статистически значимое влияние на количество заказов у пользователей и увеличивает число заказов, но не влияет на средний чек и общую выручку.